<a href="https://colab.research.google.com/github/AlexEskin/Templates/blob/master/Recognizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!rm -rf crops_train
!rm -rf crops_test
!rm -rf maps_resized
!rm -rf examples

!mkdir maps_resized
!mkdir crops_train
!mkdir crops_test
!mkdir examples

import os
import cv2
import numpy as np
import os
from google.colab import files
import random
WIDTH = 224
HEIGHT = 224
SOURCE = 'MapsExample'

def resizeImages():
    path_a = SOURCE
    path_b = 'maps_resized'

    filenames = []
    for r, d, f in os.walk(path_a):
        for file in f:
            if ".png" in file:
               filenames.append(file)
    print(filenames)
    images = []
    minSizeX = None
    minSizeY = None

    for filename in filenames:
        f_a = os.path.join(path_a, filename)
        print(f_a)
        im_a = cv2.imread(f_a)
        if minSizeX != None:
            minSizeX = min(minSizeX,im_a.shape[0])
        else:
            minSizeX = im_a.shape[0]
        if minSizeY != None:
            minSizeY = min(minSizeY,im_a.shape[1])          
        else:
            minSizeY = im_a.shape[1]
    print(minSizeX,minSizeY)
    for filename in filenames:
        f_a = os.path.join(path_a, filename)
        f_b = os.path.join(path_b, filename)

        im_a = cv2.imread(f_a)
        crop_img = im_a[0:minSizeX, 0:minSizeY]
        cv2.imwrite(f_b,crop_img)

def generateCrops():
    path_a = 'maps_resized'
    path_b = 'crops_train'
    path_c = 'crops_test'
    examples = 'examples'
    filenames = []
    for r, d, f in os.walk(path_a):
        for file in f:
            filenames.append(file)
    counter = 0
    for filename in filenames:
        print(filename)
        f_a = os.path.join(path_a, filename)
        im_a = cv2.imread(f_a)
        shape = im_a.shape
        examples_counter = 0
        for i in range(0,shape[0]-WIDTH-1,32):
            for j in range(0,shape[1]-HEIGHT-1,16):
                path = path_b
                if not counter%7:
                    path = path_c       
                submap = os.path.join(path, filename)
                try:
                  os.mkdir(submap)
                except:
                  pass
                submap = os.path.join(submap,str(counter)+".png")

                submap_image = im_a[i:i+WIDTH,j:j+HEIGHT]
                cv2.imwrite(submap,submap_image)
                if examples_counter < 32:
                    expath = os.path.join(examples, filename)
                    try:
                        if not examples_counter:
                            os.mkdir(expath)
                    except Exception as e:
                        pass
                    exmap = os.path.join(expath,str(counter)+".png")
                    cv2.imwrite(exmap,submap_image)
                    examples_counter = examples_counter + 1
                counter += 1

"""Download maps"""

!rm -rf MapsExample

!git clone https://github.com/AlexEskin/MapsExample.git

!ls MapsExample

resizeImages()
generateCrops()


In [0]:
import tensorflow as tf
import numpy as np
import datetime
import distutils
if distutils.version.LooseVersion(tf.__version__) < '1.14':
    raise Exception('This notebook is compatible with TensorFlow 1.14 or higher, for TensorFlow 1.13 or lower please use the previous version at https://github.com/tensorflow/tpu/blob/r1.13/tools/colab/fashion_mnist.ipynb')

import pathlib

import numpy as np

CLASS_CNT = None

def generate_dataset(dir):
  global CLASS_CNT
  data_root = pathlib.Path(dir)
  label_names = sorted(item.name for item in data_root.glob('*/') if item.is_dir())
  if CLASS_CNT == None:
    CLASS_CNT = len(label_names)
    
  all_image_paths = list(data_root.glob('*/*'))
  all_image_paths = [str(path) for path in all_image_paths]
  random.shuffle(all_image_paths)

  label_names = sorted(item.name for item in data_root.glob('*/') if item.is_dir())
  label_to_index = dict((name, index) for index,name in enumerate(label_names))

  all_image_labels = [label_to_index[pathlib.Path(path).parent.name]
                      for path in all_image_paths]
  print("First 10 labels indices: ", all_image_labels[:10])

  return (all_image_paths,all_image_labels)

dataset = generate_dataset("crops_train")
test_dataset = generate_dataset("crops_test")
   
def t_generator(n):
  num = 0
  while num < n:
    id = random.randint(0,len(test_dataset[0])-1)
    img = cv2.imread(test_dataset[0][id],cv2.IMREAD_GRAYSCALE)
    img = np.expand_dims(img, -1).astype(np.float32)
    yield img*(1/255.0), test_dataset[1][id]
    num += 1
    

input_data = dataset[0][0]

input_data = cv2.imread(input_data,cv2.IMREAD_GRAYSCALE).astype(np.float32)
input_data = np.expand_dims(input_data, -1)
input_data_shape = input_data.shape


In [0]:

def identity_block(input_tensor, kernel_size, filters, stage, block):
    filters1, filters2, filters3 = filters
    #if backend.image_data_format() == 'channels_last':
    bn_axis = 3
    #else:
    #    bn_axis = 1
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'

    x = tf.keras.layers.Conv2D(filters1, (1, 1),
                      kernel_initializer='he_normal',
                      name=conv_name_base + '2a')(input_tensor)
    x = tf.keras.layers.BatchNormalization(axis=bn_axis, name=bn_name_base + '2a')(x)
    x = tf.keras.layers.Activation('relu')(x)

    x = tf.keras.layers.Conv2D(filters2, kernel_size,
                      padding='same',
                      kernel_initializer='he_normal',
                      name=conv_name_base + '2b')(x)
    x = tf.keras.layers.BatchNormalization(axis=bn_axis, name=bn_name_base + '2b')(x)
    x = tf.keras.layers.Activation('relu')(x)

    x = tf.keras.layers.Conv2D(filters3, (1, 1),
                      kernel_initializer='he_normal',
                      name=conv_name_base + '2c')(x)
    x = tf.keras.layers.BatchNormalization(axis=bn_axis, name=bn_name_base + '2c')(x)

    x = tf.keras.layers.add([x, input_tensor])
    x = tf.keras.layers.Activation('relu')(x)
    return x


def conv_block(input_tensor,
               kernel_size,
               filters,
               stage,
               block,
               strides=(2, 2)):

    filters1, filters2, filters3 = filters
    bn_axis = 3
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'

    x = tf.keras.layers.Conv2D(filters1, (1, 1), strides=strides,
                      kernel_initializer='he_normal',
                      name=conv_name_base + '2a')(input_tensor)
    x = tf.keras.layers.BatchNormalization(axis=bn_axis, name=bn_name_base + '2a')(x)
    x = tf.keras.layers.Activation('relu')(x)

    x = tf.keras.layers.Conv2D(filters2, kernel_size, padding='same',
                      kernel_initializer='he_normal',
                      name=conv_name_base + '2b')(x)
    x = tf.keras.layers.BatchNormalization(axis=bn_axis, name=bn_name_base + '2b')(x)
    x = tf.keras.layers.Activation('relu')(x)

    x = tf.keras.layers.Conv2D(filters3, (1, 1),
                      kernel_initializer='he_normal',
                      name=conv_name_base + '2c')(x)
    x = tf.keras.layers.BatchNormalization(axis=bn_axis, name=bn_name_base + '2c')(x)

    shortcut = tf.keras.layers.Conv2D(filters3, (1, 1), strides=strides,
                             kernel_initializer='he_normal',
                             name=conv_name_base + '1')(input_tensor)
    shortcut = tf.keras.layers.BatchNormalization(
        axis=bn_axis, name=bn_name_base + '1')(shortcut)

    x = tf.keras.layers.add([x, shortcut])
    x = tf.keras.layers.Activation('relu')(x)
    return x


def create_model_ResNet50(input_shape=(224,224,1)):
    img_input = tf.keras.layers.Input(shape=input_shape)
    x = tf.keras.layers.ZeroPadding2D(padding=(3, 3), name='conv1_pad')(img_input)
    x = tf.keras.layers.Conv2D(64, (7, 7),
                      strides=(2, 2),
                      padding='valid',
                      kernel_initializer='he_normal',
                      name='conv1')(x)
    x = tf.keras.layers.BatchNormalization(axis = 3,name='bn_conv1')(x)
    x = tf.keras.layers.Activation('relu')(x)
    x = tf.keras.layers.ZeroPadding2D(padding=(1, 1), name='pool1_pad')(x)
    x = tf.keras.layers.MaxPooling2D((3, 3), strides=(2, 2))(x)

    x = conv_block(x, 3, [64, 64, 256], stage=2, block='a', strides=(1, 1))
    x = identity_block(x, 3, [64, 64, 256], stage=2, block='b')
    x = identity_block(x, 3, [64, 64, 256], stage=2, block='c')

    x = conv_block(x, 3, [128, 128, 512], stage=3, block='a')
    x = identity_block(x, 3, [128, 128, 512], stage=3, block='b')
    x = identity_block(x, 3, [128, 128, 512], stage=3, block='c')
    x = identity_block(x, 3, [128, 128, 512], stage=3, block='d')

    x = conv_block(x, 3, [256, 256, 1024], stage=4, block='a')
    x = identity_block(x, 3, [256, 256, 1024], stage=4, block='b')
    x = identity_block(x, 3, [256, 256, 1024], stage=4, block='c')
    x = identity_block(x, 3, [256, 256, 1024], stage=4, block='d')
    x = identity_block(x, 3, [256, 256, 1024], stage=4, block='e')
    x = identity_block(x, 3, [256, 256, 1024], stage=4, block='f')

    x = conv_block(x, 3, [512, 512, 2048], stage=5, block='a')
    x = identity_block(x, 3, [512, 512, 2048], stage=5, block='b')
    x = identity_block(x, 3, [512, 512, 2048], stage=5, block='c')
    x = tf.keras.layers.Flatten()(x)
    x = tf.keras.layers.Dense(CLASS_CNT,activation='softmax')(x)
    model = tf.keras.models.Model(img_input, x, name='resnet50')
    return model


In [0]:
import os

strategy = None
if 'COLAB_TPU_ADDR' in os.environ:
  resolver = tf.contrib.cluster_resolver.TPUClusterResolver('grpc://' + os.environ['COLAB_TPU_ADDR'])
  tf.contrib.distribute.initialize_tpu_system(resolver)
  strategy = tf.contrib.distribute.TPUStrategy(resolver)

def create_model_vgg19():
  model = tf.keras.models.Sequential()
  drop = 0.5
  model.add(tf.keras.layers.BatchNormalization(input_shape=input_data_shape))
  model.add(tf.keras.layers.Conv2D(64, (3,3), padding='same', activation='relu'))
  model.add(tf.keras.layers.Conv2D(64, (3,3), padding='same', activation='relu'))
  # model.add(tf.keras.layers.Dropout(drop))
  #model.add(tf.keras.layers.BatchNormalization())

  model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2,2)))
    
  model.add(tf.keras.layers.Conv2D(128, (3,3), padding='same', activation='relu'))
  model.add(tf.keras.layers.Conv2D(128, (3,3), padding='same', activation='relu'))
  # model.add(tf.keras.layers.Dropout(drop))
  # model.add(tf.keras.layers.BatchNormalization())

  model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2,2)))
  
  model.add(tf.keras.layers.Conv2D(256, (3,3), padding='same', activation='relu'))
  model.add(tf.keras.layers.Conv2D(256, (3,3), padding='same', activation='relu'))
  model.add(tf.keras.layers.Conv2D(256, (3,3), padding='same', activation='relu'))
  model.add(tf.keras.layers.Conv2D(256, (3,3), padding='same', activation='relu'))
  # model.add(tf.keras.layers.Dropout(drop))
  # model.add(tf.keras.layers.BatchNormalization())

  model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2,2)))
  
  model.add(tf.keras.layers.Conv2D(512, (3,3), padding='same', activation='relu'))
  model.add(tf.keras.layers.Conv2D(512, (3,3), padding='same', activation='relu'))
  model.add(tf.keras.layers.Conv2D(512, (3,3), padding='same', activation='relu'))
  model.add(tf.keras.layers.Conv2D(512, (3,3), padding='same', activation='relu'))
  # model.add(tf.keras.layers.Dropout(drop))
  # model.add(tf.keras.layers.BatchNormalization())

  model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2,2)))
  
  model.add(tf.keras.layers.Conv2D(512, (3,3), padding='same', activation='relu'))
  model.add(tf.keras.layers.Conv2D(512, (3,3), padding='same', activation='relu'))
  model.add(tf.keras.layers.Conv2D(512, (3,3), padding='same', activation='relu'))
  model.add(tf.keras.layers.Conv2D(512, (3,3), padding='same', activation='relu'))
  # model.add(tf.keras.layers.Dropout(drop))
  # model.add(tf.keras.layers.BatchNormalization())

  model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2,2)))

  model.add(tf.keras.layers.Flatten())
  model.add(tf.keras.layers.Dense(4096, activation='relu'))
  model.add(tf.keras.layers.BatchNormalization())
  model.add(tf.keras.layers.Dropout(drop))

  model.add(tf.keras.layers.Dense(4096, activation='relu'))
  model.add(tf.keras.layers.BatchNormalization())
  model.add(tf.keras.layers.Dropout(drop))

  #model.add(tf.keras.layers.Dense(256))
  #model.add(tf.keras.layers.Activation('elu'))  
  #model.add(tf.keras.layers.BatchNormalization())
  #model.add(tf.keras.layers.Dropout(drop))
  
  model.add(tf.keras.layers.Dense(CLASS_CNT,activation='softmax'))
  model.summary()
  return model

def create_model():
  model = tf.keras.models.Sequential()
  drop = 0.5
  model.add(tf.keras.layers.BatchNormalization(input_shape=input_data_shape))
  model.add(tf.keras.layers.Conv2D(64, (9, 9), padding='same', activation='elu'))
  model.add(tf.keras.layers.Conv2D(128, (9, 9), padding='same', activation='elu'))
  model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2,2)))
  model.add(tf.keras.layers.Dropout(drop))

  model.add(tf.keras.layers.BatchNormalization(input_shape=input_data_shape))
  model.add(tf.keras.layers.Conv2D(96, (7, 7), padding='same', activation='elu'))
  model.add(tf.keras.layers.Conv2D(192, (7, 7), padding='same', activation='elu'))
  model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
  model.add(tf.keras.layers.Dropout(drop))

  model.add(tf.keras.layers.BatchNormalization(input_shape=input_data_shape))
  model.add(tf.keras.layers.Conv2D(128, (5, 5), padding='same', activation='elu'))
  model.add(tf.keras.layers.Conv2D(256, (5, 5), padding='same', activation='elu'))
  model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2,2)))
  model.add(tf.keras.layers.Dropout(drop))

  model.add(tf.keras.layers.BatchNormalization(input_shape=input_data_shape))
  model.add(tf.keras.layers.Conv2D(128, (3, 3), padding='same', activation='elu'))
  model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2,2)))
  model.add(tf.keras.layers.Dropout(drop))
  
  model.add(tf.keras.layers.Flatten())
  model.add(tf.keras.layers.Dense(1024))
  model.add(tf.keras.layers.Activation('elu'))
  model.add(tf.keras.layers.Dense(256))
  model.add(tf.keras.layers.Activation('elu'))  
  model.add(tf.keras.layers.Dropout(drop))
  model.add(tf.keras.layers.Dense(CLASS_CNT,activation='softmax'))
  model.summary()
  return model

if strategy != None:
  with strategy.scope():
    model = create_model_ResNet50()#create_model_vgg19()
    model.compile(
        #optimizer=tf.keras.optimizers.Adadelta( ),
        optimizer=tf.keras.optimizers.SGD(momentum=0.9),
         
        loss='sparse_categorical_crossentropy',
        metrics=['sparse_categorical_accuracy'])
else:
    model = create_model_ResNet50()#create_model_vgg19()
    model.compile(
        optimizer=tf.keras.optimizers.Adadelta( ),
        loss='sparse_categorical_crossentropy',
        metrics=['sparse_categorical_accuracy'])

In [0]:
from google.colab import drive
drive.mount('/content/drive')
try:
  print("Loading...")
  model.load_weights('/content/drive/My Drive/resnet50.h5')
  print("Loading... Ready")
except:
  print("Unable to load weights...")
  pass

In [0]:
def validate(model):
  result = model.predict(np.array(test_xy[0]).astype(np.float32))
  pos = 0
  for i in range(len(result)):
    test_y = result[i].argmax()
    valid_y = test_xy[1][i]
    #print(test_y, valid_y, test_y==valid_y)
    pos += (test_y==valid_y)
  print("validation status :",pos/len(result))
  print(datetime.datetime.now())    

In [0]:
def get_data(n):
  num = 0
  x = []
  y = []
  while num < n:
    id = random.randint(0,len(dataset[0])-1)
    darkness = random.randint(128,255)/255.0
    img = cv2.imread(dataset[0][id],cv2.IMREAD_GRAYSCALE)
    img = np.expand_dims(img, -1).astype(np.float32)
    x.append(img*(1/255.0)*darkness)
    y.append(dataset[1][id])
    num += 1
  return np.array(x),np.array(y)


In [0]:
validation_generator = t_generator(400)
test_xy = list(zip(*validation_generator))
batch_size = 256
data_length = ((10000)//batch_size)*batch_size
print("Data lenght = ", data_length)
for i in range(20):
  print("train ready")
  x,y = get_data(data_length)
  print("fitting")
  history = model.fit(
      x,y,
      epochs=100,
      batch_size=batch_size
  )
  validate(model)
  model.save_weights('/content/drive/My Drive/resnet50.h5', overwrite=True)
